U ovoj svesci je odrađena selekcija i evaluacija modela. Koristiće se tokenizacija ne sekvence kao priprema za model sa pritreniram ugneždavanjem,

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, GRU, Flatten, Dropout
from tensorflow.keras import preprocessing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

In [3]:
data = pd.read_csv('data/data_longer.csv')

In [4]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
data.shape

(223453, 9)

### Podela na trening, test i validacioni skup

In [6]:
# Ovo je prepisano iz prethodne sveske

train_size = 159571 - 96
test_size = 63978
validation_size = int(0.2*train_size)
train_size = train_size - validation_size

train_size, test_size, validation_size

(127580, 63978, 31895)

In [7]:
validation_data = data[:validation_size]
train_data = data[validation_size:train_size+validation_size]
test_data = data[train_size+validation_size : ]

In [8]:
train_data.shape[0], test_data.shape[0], validation_data.shape[0]

(127580, 63978, 31895)

### 1. Tokenizovati tekst u sekvence

Instance će biti podeljene na reči, izdvojiće se neki broj najfrekventnijih reči, a onda će biti pretvorene u vektore reči sekvenci

In [9]:
# uzima se recimo 20000 najfrekventinijh reci
max_features = 20000

# max duzina komentara (broj reci)
max_len = 200

Izdvajaju se pojedinačne reči tj. tokeni zadatog teksta i konstruise se vokabular. 

In [10]:
tokenizer = Tokenizer(num_words=max_features)

In [11]:
tokenizer.fit_on_texts(train_data['comment_text'])

In [14]:
# vokabular:

word_index = tokenizer.word_index

# broj jedinstvenih tokena:
len(word_index)

187048

In [13]:
# cuvanje tokenizera
with open('data/tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)

In [15]:
train_sequences = tokenizer.texts_to_sequences(train_data['comment_text'])
X_train = preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
y_train = train_data['target']

test_sequences = tokenizer.texts_to_sequences(test_data['comment_text'])
X_test = preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_len)
y_test = test_data['target']

validation_sequences = tokenizer.texts_to_sequences(validation_data['comment_text'])
X_validation = preprocessing.sequence.pad_sequences(validation_sequences, maxlen=max_len)
y_validation = validation_data['target']

Sada su podaci pripremljeni za model